In [1]:
import data
import eval
import model
import preprocess
import postprocess

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

First off, the steps taken to fully load in the data set.

Step 1: load the raw text files and their id's into a dataframe

Step 2: if the texts are not in English, translate them

Step 3: concatenate the text files into a single dataset

Step 4: load in the labels

Step 5: merge the text and labels data sets

This can take quite a while, due to the translation step, skip this for the next cell, once we have all the data correctly saved as a csv, so we just load in that csv again. 

In [3]:
# load english text data
df_text_en = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/raw-documents")

# load and translate hindi and portugese text
# df_text_hi = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/raw-documents", True, source_lang="hi")
# df_text_pt = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/raw-documents", True, source_lang = "pt")
# df_text = pd.concat([df_text_en, df_text_hi, df_text_pt])
df_text = df_text_en

# load all the labels, these are all in english
df_labels_en = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-2-annotations.txt")
# df_labels_hi = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/subtask-2-annotations.txt")
# df_labels_pt = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/subtask-2-annotations.txt")
# df_labels = pd.concat([df_labels_en, df_labels_hi, df_labels_pt])
df_labels = df_labels_en

dom_mlb, sub_mlb, dom_narr_enc, sub_narr_enc, narr_df = preprocess.encode_labels(df_labels)

df = pd.merge(df_text, narr_df, on="id")

df.to_csv("data.csv")

Data length:  200
Labels length: 200


first load the data set, and save it as a csv?

In [4]:
# df = pd.read_csv("data.csv")
# dom_mlb, sub_mlb, dom_narr_enc, sub_narr_enc = preprocess.encode_labels(df)
# print(df.shape)

Train it on the sub classes

In [8]:
vec = TfidfVectorizer(min_df=0.02,max_df=0.75)
X = vec.fit_transform(df.text)
y = sub_narr_enc


In [6]:
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=sub_mlb.classes_))

Skipped fold 31, labels invalid
Skipped fold 43, labels invalid
Skipped fold 58, labels invalid
Skipped fold 61, labels invalid
Skipped fold 67, labels invalid
Skipped fold 74, labels invalid
Skipped fold 81, labels invalid
Skipped fold 142, labels invalid
Skipped fold 143, labels invalid
Skipped fold 161, labels invalid
Skipped fold 174, labels invalid
Skipped fold 180, labels invalid
Skipped fold 189, labels invalid
Skipped fold 193, labels invalid
                                                                                                            precision    recall  f1-score   support

                                                       CC: Climate change is beneficial: CO2 is beneficial       0.00      0.00      0.00         0
                                                           CC: Controversy about green technologies: Other       0.00      0.00      0.00         2
                                      CC: Controversy about green technologies: Renewable energy is 

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained on the dom classes:

In [7]:
y = dom_narr_enc
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=dom_mlb.classes_))

Skipped fold 67, labels invalid
Skipped fold 174, labels invalid
                                                        precision    recall  f1-score   support

                      CC: Climate change is beneficial       0.00      0.00      0.00         0
              CC: Controversy about green technologies       0.00      0.00      0.00         3
                     CC: Criticism of climate movement       0.00      0.00      0.00        10
                     CC: Criticism of climate policies       0.00      0.00      0.00        10
         CC: Criticism of institutions and authorities       0.00      0.00      0.00        19
                        CC: Downplaying climate change       0.00      0.00      0.00         1
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         0
 CC: Hidden plots by secret schemes of powerful groups       0.00      0.00      0.00         6
          CC: Questioning the measurements and science       0.00     

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
